In [1]:
# importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Kerala Bus Detalis

In [2]:
# read the csv file
df=pd.read_csv("df_k.csv")
df

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [35]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [36]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_1 = pd.DataFrame(data)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_1.csv"
df_buses_1.to_csv(path,index=False)

In [38]:
import pandas as pd

# 1. Load DataFrame
df_buses_1 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_1.csv")

# 2. Filter for KSRTC Buses
df_ksrtc = df_buses_1[
    df_buses_1['Bus_name'].str.contains('KSRTC', case=False, na=False) |
    df_buses_1['Bus_type'].str.contains('KSRTC', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered KSRTC Buses:")
print(df_ksrtc.to_string(index=False))

# 4. Save the filtered DataFrame to a new CSV file
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\ksrtc_buses.csv"
df_ksrtc.to_csv(filtered_path, index=False)

print(f"Filtered KSRTC buses saved to: {filtered_path}")


Filtered KSRTC Buses:
             Bus_name                                  Bus_type Start_time End_time Total_duration    Price    Seats_Available Ratings                                                        Route_link                      Route_name
  KSRTC (Kerala) - 97         Swift Deluxe Non AC Air Bus (2+2)      16:00    00:45        08h 45m  INR 567  7 Seats available 2.8\n22          https://www.redbus.in/bus-tickets/bangalore-to-kozhikode          Bangalore to Kozhikode
 KSRTC (Kerala) - 380         Swift Deluxe Non AC Air Bus (2+2)      16:15    01:00        08h 45m  INR 567  5 Seats available  4.7\n2          https://www.redbus.in/bus-tickets/bangalore-to-kozhikode          Bangalore to Kozhikode
  KSRTC (Kerala) - 51         Swift Deluxe Non AC Air Bus (2+2)      19:00    04:30        09h 30m  INR 640 19 Seats available 4.0\n48          https://www.redbus.in/bus-tickets/bangalore-to-kozhikode          Bangalore to Kozhikode
 KSRTC (Kerala) - 789            Super Fast No

# Andhra Bus Details

In [5]:
# read the csv file
df_1=pd.read_csv("df_A.csv")
df_1

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
5,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...
6,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...
7,Kadapa to Bangalore,https://www.redbus.in/bus-tickets/kadapa-to-ba...
8,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
9,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...


In [6]:
#retrive the bus details
driver_A = webdriver.Chrome()
Bus_names_A = []
Bus_types_A = []
Start_Time_A = []
End_Time_A = []
Ratings_A = []
Total_Duration_A = []
Prices_A = []
Seats_Available_A = []
Route_names = []
Route_links = []

for i,r in df_1.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_A.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_A.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_A.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_A.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_A).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  # Adjust sleep time as needed
        
        new_page_source = driver_A.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_A.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_A.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_A.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_A.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_A.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_A.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_A.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_A.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_A.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_A.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_A.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_A.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_A.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_A.append(ratings.text)
    for price_elem in price:
        Prices_A.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_A.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [7]:
# from list to convert data frame
data_1 = {
    'Bus_name': Bus_names_A,
    'Bus_type': Bus_types_A,
    'Start_time': Start_Time_A,
    'End_time': End_Time_A,
    'Total_duration': Total_Duration_A,
    'Price': Prices_A,
    "Seats_Available":Seats_Available_A,
    "Ratings":Ratings_A,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_2 = pd.DataFrame(data_1)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_2.csv"
df_buses_2.to_csv(path,index=False)

In [8]:
df_buses_2

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:25,06:15,06h 50m,INR 1209,13 Seats available,4.7\n857,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad
1,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:45,05:45,06h 00m,INR 1139,14 Seats available,4.6\n1079,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad
2,IntrCity SmartBus,AC Sleeper (2+1),23:40,06:30,06h 50m,INR 1519,11 Seats available,4.7\n326,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad
3,AdSai RK Travels,Non A/C Seater / Sleeper (2+1),22:10,05:30,07h 20m,INR 830,7 Seats available,3.8\n131,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad
4,FRESHBUS,Electric A/C Seater (2+2),22:50,05:00,06h 10m,1299,17 Seats available,4.6\n564,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad
...,...,...,...,...,...,...,...,...,...,...
405,,,,,,,,,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada
406,,,,,,,,,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada
407,,,,,,,,,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada
408,,,,,,,,,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada


In [39]:
import pandas as pd

# Load DataFrame
df_buses_2 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_2.csv")

# Filter for APSRTC Buses
df_apsrtc = df_buses_2[
    df_buses_2['Bus_name'].str.contains('APSRTC', case=False, na=False) |
    df_buses_2['Bus_type'].str.contains('APSRTC', case=False, na=False)
]

# Display the filtered DataFrame
print("Filtered APSRTC Buses:")
print(df_apsrtc.to_string(index=False))

# Save the filtered DataFrame to a new CSV file
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\apsrtc_buses.csv"
df_apsrtc.to_csv(filtered_path, index=False)

print(f"Filtered APSRTC buses saved to: {filtered_path}")


Filtered APSRTC Buses:
      Bus_name                                       Bus_type Start_time End_time Total_duration   Price    Seats_Available  Ratings                                                             Route_link                             Route_name
 APSRTC - 2819                             INDRA(A.C. Seater)      14:30    17:30        03h 00m INR 340   1 Seat available 3.4\n105            https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam              Kakinada to Visakhapatnam
 APSRTC - 2669         SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)      15:00    18:00        03h 00m INR 281 14 Seats available  3.0\n39            https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam              Kakinada to Visakhapatnam
 APSRTC - 2532         SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)      15:10    19:45        04h 35m INR 281 34 Seats available   2.3\n3            https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam              Kakinada to Visakhapatnam
 APSR

# Telugana Bus Details

In [9]:
# read the csv file
df_2=pd.read_csv("df_T.csv")
df_2

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [10]:
# retrive the bus details
driver_T = webdriver.Chrome()
Bus_names_T = []
Bus_types_T = []
Start_Time_T = []
End_Time_T = []
Ratings_T = []
Total_Duration_T = []
Prices_T = []
Seats_Available_T = []
Route_names = []
Route_links = []

for i,r in df_2.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_T.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_T.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_T.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_T.page_source
        
        ActionChains(driver_T).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_T.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_T.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_T.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_T.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_T.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_T.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_T.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_T.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_T.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_T.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_T.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_T.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_T.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_T.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_T.append(ratings.text)
    for price_elem in price:
        Prices_T.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_T.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [11]:
# from list to convert data frame
data_3 = {
    'Bus_name': Bus_names_T,
    'Bus_type': Bus_types_T,
    'Start_time': Start_Time_T,
    'End_time': End_Time_T,
    'Total_duration': Total_Duration_T,
    'Price': Prices_T,
    "Seats_Available":Seats_Available_T,
    "Ratings":Ratings_T,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_3 = pd.DataFrame(data_3)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_3.csv"
df_buses_3.to_csv(path,index=False)

In [12]:
df_buses_3

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,TGSRTC - 8977,Super Luxury (Non AC Seater 2+2 Push Back),13:50,17:45,03h 55m,INR 320,17 Seats available,4.2\n18,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
1,TGSRTC - 64020,RAJDHANI (A.C. Semi Sleeper),14:00,18:00,04h 00m,INR 389,8 Seats available,3.9\n64,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
2,TGSRTC - 64863,Super Luxury (Non AC Seater 2+2 Push Back),14:30,18:30,04h 00m,INR 320,21 Seats available,4.0\n3,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
3,TGSRTC - 8974,Super Luxury (Non AC Seater 2+2 Push Back),14:40,18:30,03h 50m,INR 320,24 Seats available,3.1\n15,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
4,TGSRTC - 8665,LAHARI A/C SLEEPER CUM SEATER,15:00,20:00,05h 00m,INR 389,14 Seats available,4.4\n72,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
...,...,...,...,...,...,...,...,...,...,...
196,Kamakshi Tours And Travels,Non A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,INR 450,18 Seats available,4.4\n252,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
197,Chandu Travels,A/C Sleeper (2+1),22:00,05:45,07h 45m,589,16 Seats available,4.3\n160,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
198,Kaveri Travels,Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1),21:30,05:45,08h 15m,INR 810,16 Seats available,4.1\n163,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
199,,,,,,,,,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole


In [42]:
import pandas as pd

# 1. Load DataFrame
df_buses_3 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_3.csv")

# 2. Filter for TGSRTC Buses
df_tgsrtc = df_buses_3[
    df_buses_3['Bus_name'].str.contains('TGSRTC', case=False, na=False) |
    df_buses_3['Bus_type'].str.contains('TGSRTC', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered TGSRTC Buses:")
print(df_tgsrtc.to_string(index=False))  # Display in a formatted way

# 4. Save the Filtered DataFrame to a New CSV File
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\tgsrtc_buses.csv"
df_tgsrtc.to_csv(filtered_path, index=False)

print(f"Filtered TGSRTC buses saved to: {filtered_path}")


Filtered TGSRTC Buses:
      Bus_name                                   Bus_type Start_time End_time Total_duration   Price    Seats_Available Ratings                                                Route_link              Route_name
 TGSRTC - 8977 Super Luxury (Non AC Seater 2+2 Push Back)      13:50    17:45        03h 55m INR 320 17 Seats available 4.2\n18    https://www.redbus.in/bus-tickets/khammam-to-hyderabad    Khammam to Hyderabad
TGSRTC - 64020               RAJDHANI (A.C. Semi Sleeper)      14:00    18:00        04h 00m INR 389  8 Seats available 3.9\n64    https://www.redbus.in/bus-tickets/khammam-to-hyderabad    Khammam to Hyderabad
TGSRTC - 64863 Super Luxury (Non AC Seater 2+2 Push Back)      14:30    18:30        04h 00m INR 320 21 Seats available  4.0\n3    https://www.redbus.in/bus-tickets/khammam-to-hyderabad    Khammam to Hyderabad
 TGSRTC - 8974 Super Luxury (Non AC Seater 2+2 Push Back)      14:40    18:30        03h 50m INR 320 24 Seats available 3.1\n15    https:

# Goa Bus Details

In [13]:
# read the csv file
df_3=pd.read_csv("df_G.csv")
df_3

,Route_name,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
4,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
5,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
6,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
7,Goa to Pandharpur,https://www.redbus.in/bus-tickets/goa-to-pandh...
8,Solapur to Goa,https://www.redbus.in/bus-tickets/solapur-to-goa
9,Calangute (goa) to Goa Airport,https://www.redbus.in/bus-tickets/calangute-go...


In [14]:
# retrive the bus details
driver_G = webdriver.Chrome()
Bus_names_G = []
Bus_types_G = []
Start_Time_G = []
End_Time_G = []
Ratings_G = []
Total_Duration_G = []
Prices_G = []
Seats_Available_G = []
Route_names = []
Route_links = []

for i,r in df_3.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_G.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_G.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
   
    time.sleep(2)
    scrolling = True
    while scrolling:
        old_page_source = driver_G.page_source
        
        ActionChains(driver_G).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_G.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_G.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_G.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_G.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_G.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_G.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_G.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_G.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_G.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_G.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_G.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_G.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_G.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_G.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_G.append(ratings.text)
    for price_elem in price:
        Prices_G.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_G.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [15]:
# from list to convert data frame
data_4 = {
    'Bus_name': Bus_names_G,
    'Bus_type': Bus_types_G,
    'Start_time': Start_Time_G,
    'End_time': End_Time_G,
    'Total_duration': Total_Duration_G,
    'Price': Prices_G,
    "Seats_Available":Seats_Available_G,
    "Ratings":Ratings_G,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_4 = pd.DataFrame(data_4)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_4.csv"
df_buses_4.to_csv(path,index=False)

In [16]:
df_buses_4 

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,05:50,10h 35m,INR 1000,3 Seats available,3.8\n333,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
1,Ashray Travels,Non A/C Seater / Sleeper (2+1),21:15,08:31,11h 16m,INR 500,15 Seats available,4.4\n502,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
2,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:15,07:55,10h 40m,INR 600,22 Seats available,4.4\n473,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
3,AdIndumati Travels,A/C Sleeper (2+1),22:20,08:30,10h 10m,855,23 Seats available,4.1\n382,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
4,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:05,08:30,11h 25m,INR 900,16 Seats available,4.4\n671,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
...,...,...,...,...,...,...,...,...,...,...
139,S R Travels,A/C Seater/Sleeper (2+1),18:45,07:00,12h 15m,INR 700,22 Seats available,2.4\n101,https://www.redbus.in/bus-tickets/solapur-to-goa,Solapur to Goa
140,New Shivbaba Tours And Travels,AC Sleeper (2+1),18:00,06:30,12h 30m,899,11 Seats available,2.9\n7,https://www.redbus.in/bus-tickets/solapur-to-goa,Solapur to Goa
141,Sharma Travels,Bharat Benz A/C Sleeper (2+1),23:00,09:00,10h 00m,INR 1300,10 Seats available,3.8,https://www.redbus.in/bus-tickets/solapur-to-goa,Solapur to Goa
142,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,17:15,01h 45m,INR 200,40 Seats available,4.1\n20,https://www.redbus.in/bus-tickets/calangute-go...,Calangute (goa) to Goa Airport


In [43]:
import pandas as pd

# 1. Load DataFrame
df_buses_4 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_4.csv")

# 2. Filter for KTCL Buses
df_ktcl = df_buses_4[
    df_buses_4['Bus_name'].str.contains('KTCL', case=False, na=False) |
    df_buses_4['Bus_type'].str.contains('KTCL', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered KTCL Buses:")
print(df_ktcl.to_string(index=False))  # Display in a formatted way

# 4. Save the Filtered DataFrame to a New CSV File
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\ktcl_buses.csv"
df_ktcl.to_csv(filtered_path, index=False)

print(f"Filtered KTCL buses saved to: {filtered_path}")


Filtered KTCL Buses:
                                             Bus_name                     Bus_type Start_time End_time Total_duration    Price    Seats_Available  Ratings                                                     Route_link                     Route_name
 Kadamba Transport Corporation Limited (KTCL) - 89821            A/C Sleeper (2+1)      19:15    05:50        10h 35m INR 1000  3 Seats available 3.8\n333                  https://www.redbus.in/bus-tickets/pune-to-goa                    Pune to Goa
 Kadamba Transport Corporation Limited (KTCL) - 89820            A/C Sleeper (2+1)      19:00    07:00        12h 00m INR 1400  2 Seats available 3.1\n305                  https://www.redbus.in/bus-tickets/goa-to-pune                    Goa to Pune
 Kadamba Transport Corporation Limited (KTCL) - 97386 Volvo AC Seater Pushback 2+2      17:05    06:00        12h 55m  INR 800  7 Seats available 3.7\n355                https://www.redbus.in/bus-tickets/mumbai-to-goa               

# Rajastan Bus Details

In [17]:
# read the csv file
df_4=pd.read_csv("df_R.csv")
df_4

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [18]:
# retrive the bus details
driver_R = webdriver.Chrome()
Bus_names_R = []
Bus_types_R = []
Start_Time_R = []
End_Time_R = []
Ratings_R = []
Total_Duration_R = []
Prices_R = []
Seats_Available_R = []
Route_names_R = []
Route_links_R = []

for i,r in df_4.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_R.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_R.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver_R.page_source
        
        ActionChains(driver_R).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_R.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_R.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_R.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_R.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_R.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_R.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_R.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_R.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_R.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_R.append(bus.text)
        Route_links_R.append(link)
        Route_names_R.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_R.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_R.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_R.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_R.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_R.append(ratings.text)
    for price_elem in price:
        Prices_R.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_R.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [19]:
# from list to convert data frame
data_5 = {
    'Bus_name': Bus_names_R,
    'Bus_type': Bus_types_R,
    'Start_time': Start_Time_R,
    'End_time': End_Time_R,
    'Total_duration': Total_Duration_R,
    'Price': Prices_R,
    "Seats_Available":Seats_Available_R,
    "Ratings":Ratings_R,
    'Route_link': Route_links_R,
    'Route_name': Route_names_R
}

df_buses_5 = pd.DataFrame(data_5)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_5.csv"
df_buses_5.to_csv(path,index=False)

In [20]:
df_buses_5

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Shree Devnarayan Travels,Bharat Benz A/C Seater (2+1),17:00,21:30,04h 30m,INR 550,16 Seats available,4.6\n287,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
1,M R Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),17:10,21:55,04h 45m,INR 485,42 Seats available,4.4\n446,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
2,Jain travels regd,A/C Seater / Sleeper (2+1),21:30,04:30,07h 00m,INR 460,37 Seats available,4.2\n246,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
3,AdKalpana Travels,NON A/C Sleeper (2+1),22:00,04:54,06h 54m,INR 450,26 Seats available,4.1\n110,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
4,Raj Travel,A/C Sleeper (2+1),22:30,05:30,07h 00m,INR 550,20 Seats available,4.2\n101,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
...,...,...,...,...,...,...,...,...,...,...
95,UPSRTC - KSN0275,Ordinary Non AC Seater 2+3,18:06,20:30,02h 24m,INR 121,52 Seats available,3.3,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)
96,Jain travels regd,NON AC Seater / Sleeper 2+1,15:45,18:30,02h 45m,INR 250,36 Seats available,3.8,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)
97,Rinku Travels,A/C Seater / Sleeper (2+1),17:30,21:00,03h 30m,333,41 Seats available,3.5,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)
98,Jain travels regd,A/C Seater / Sleeper (2+1),18:45,20:45,02h 00m,INR 300,40 Seats available,3.8,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)


In [45]:
import pandas as pd

# 1. Load DataFrame
df_buses_5 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_5.csv")

# 2. Filter for RSRTC Buses
df_rsrtc = df_buses_5[
    df_buses_5['Bus_name'].str.contains('RSRTC', case=False, na=False) |
    df_buses_5['Bus_type'].str.contains('RSRTC', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered RSRTC Buses:")
print(df_rsrtc.to_string(index=False))  # Display in a formatted way

# 4. Save the Filtered DataFrame to a New CSV File
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\rsrtc_buses.csv"
df_rsrtc.to_csv(filtered_path, index=False)

print(f"Filtered RSRTC buses saved to: {filtered_path}")


Filtered RSRTC Buses:
Empty DataFrame
Columns: [Bus_name, Bus_type, Start_time, End_time, Total_duration, Price, Seats_Available, Ratings, Route_link, Route_name]
Index: []
Filtered RSRTC buses saved to: C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\rsrtc_buses.csv


# Chandigarh Bus Details

In [51]:
# read the csv file
df_5=pd.read_csv("df_ch.csv")
df_5

,Route_name,Route_link
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
2,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
3,Ludhiana to Chandigarh,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...
5,Chandigarh to Ludhiana,https://www.redbus.in/bus-tickets/chandigarh-t...
6,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
7,Chandigarh to Vrindavan,https://www.redbus.in/bus-tickets/chandigarh-t...
8,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...


In [53]:
# Retrieve the bus details
driver_CT = webdriver.Chrome()
Bus_names_CT = []
Bus_types_CT = []
Start_Time_CT = []
End_Time_CT = []
Ratings_CT = []
Total_Duration_CT = []
Prices_CT = []
Seats_Available_CT = []
Route_names_CT = []
Route_links_CT = []

for i, r in df_5.iterrows():  # Assuming df_ch contains your route data for CTU
    link = r["Route_link"]
    routes = r["Route_name"]

    # Loop through each link
    driver_CT.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    elements = driver_CT.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver_CT.page_source
        
        ActionChains(driver_CT).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)
        
        new_page_source = driver_CT.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_CT.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")  # Update this XPath if needed
    bus_type = driver_CT.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  # Update this XPath if needed
    start_time = driver_CT.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")  # Update this XPath if needed
    end_time = driver_CT.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")  # Update this XPath if needed
    total_duration = driver_CT.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")  # Update this XPath if needed
    try:
        rating = driver_CT.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")  # Update this XPath if needed
    except:
        continue
    price = driver_CT.find_elements(By.XPATH, '//*[@class="fare d-block"]')  # Update this XPath if needed
    seats = driver_CT.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")  # Update this XPath if needed

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_CT.append(bus.text)
        Route_links_CT.append(link)
        Route_names_CT.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_CT.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_CT.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_CT.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_CT.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_CT.append(ratings.text)
    for price_elem in price:
        Prices_CT.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_CT.append(seats_elem.text)

print("Successfully Completed")


Successfully Completed


In [55]:
data_6 = {
    'Bus_name': Bus_names_CT,
    'Bus_type': Bus_types_CT,
    'Start_time': Start_Time_CT,
    'End_time': End_Time_CT,
    'Total_duration': Total_Duration_CT,
    'Price': Prices_CT,
    "Seats_Available": Seats_Available_CT,
    "Ratings": Ratings_CT,
    'Route_link': Route_links_CT,
    'Route_name': Route_names_CT
}

# Convert the dictionary to a DataFrame
df_buses_6 = pd.DataFrame(data_6)

# Specify the path where you want to save the CSV file
path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_6.csv"  # Adjusted for CTU data

# Convert the DataFrame to a CSV file
df_buses_6.to_csv(path, index=False)

In [56]:
df_buses_6

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Zimindara Travels,AC Sleeper (2+1),20:30,23:00,02h 30m,INR 674,29 Seats available,4.1,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
1,City Land Travels,A/C Seater/Sleeper (2+1),22:20,00:20,02h 00m,INR 600,27 Seats available,3.5,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
2,Om Sidh Express,Volvo A/C (2+2),20:20,23:00,02h 40m,468,19 Seats available,3.5,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
3,KN Nehra Travels,A/C Seater/Sleeper (2+1),22:50,01:15,02h 25m,INR 699,32 Seats available,2.9,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
4,KN Nehra Travels,A/C Seater/Sleeper (2+1),22:50,01:30,02h 40m,INR 699,32 Seats available,2.9,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
...,...,...,...,...,...,...,...,...,...,...
87,Shree Mahaveer Travels Jaipur,Bharat Benz A/C Seater /Sleeper (2+1),22:00,03:15,05h 15m,INR 499,17 Seats available,3.6\n31,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haridwar
88,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),Bharat Benz A/C Seater /Sleeper (2+1),23:15,05:35,06h 20m,809,22 Seats available,3.6,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haridwar
89,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),Bharat Benz A/C Seater /Sleeper (2+1),23:30,05:35,06h 05m,809,22 Seats available,4.9\n4,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haridwar
90,Vijay Tour and Travels,,23:30,04:30,05h 00m,INR 549,17 Seats available,4.1,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haridwar


In [58]:
import pandas as pd

# 1. Load DataFrame
df_buses_6 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_6.csv")

# 2. Filter for CTU Buses
df_ctu = df_buses_6[
    df_buses_6['Bus_name'].str.contains('CTU', case=False, na=False) |
    df_buses_6['Bus_type'].str.contains('CTU', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered CTU Buses:")
print(df_ctu.to_string(index=False))  # Display in a formatted way

# 4. Save the Filtered DataFrame to a New CSV File
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\ctu_buses.csv"
df_ctu.to_csv(filtered_path, index=False)

print(f"Filtered CTU buses saved to: {filtered_path}")



Filtered CTU Buses:
                                       Bus_name          Bus_type Start_time End_time Total_duration     Price    Seats_Available  Ratings                                                                Route_link                                Route_name
Chandigarh Transport Undertaking (CTU) - 165806 HVAC Seater (2+3)      16:35    19:20        02h 45m INR 181.4 43 Seats available  4.1\n50                  https://www.redbus.in/bus-tickets/ludhiana-to-chandigarh                    Ludhiana to Chandigarh
Chandigarh Transport Undertaking (CTU) - 165807 HVAC Seater (2+3)      16:59    19:44        02h 45m INR 181.4 45 Seats available  4.5\n90                  https://www.redbus.in/bus-tickets/chandigarh-to-ludhiana                    Chandigarh to Ludhiana
Chandigarh Transport Undertaking (CTU) - 165844 HVAC Seater (2+3)      16:18    20:48        04h 30m INR 456.2 44 Seats available 4.2\n138 https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh Ha

In [60]:
df_ctu

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
16,Chandigarh Transport Undertaking (CTU) - 165806,HVAC Seater (2+3),16:35,19:20,02h 45m,INR 181.4,43 Seats available,4.1\n50,https://www.redbus.in/bus-tickets/ludhiana-to-...,Ludhiana to Chandigarh
26,Chandigarh Transport Undertaking (CTU) - 165807,HVAC Seater (2+3),16:59,19:44,02h 45m,INR 181.4,45 Seats available,4.5\n90,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Ludhiana
31,Chandigarh Transport Undertaking (CTU) - 165844,HVAC Seater (2+3),16:18,20:48,04h 30m,INR 456.2,44 Seats available,4.2\n138,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Chandigarh
36,Chandigarh Transport Undertaking (CTU) - 165938,HVAC Seater (2+3),20:00,05:45,09h 45m,INR 634.7,47 Seats available,3.8\n242,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Vrindavan
65,Chandigarh Transport Undertaking (CTU) - 165747,HVAC Seater (2+3),17:12,21:47,04h 35m,INR 308,47 Seats available,3.6\n15,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Shimla
66,Chandigarh Transport Undertaking (CTU) - 165825,HVAC Seater (2+3),20:08,05:08,09h 00m,INR 611.4,47 Seats available,4.0\n62,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Baijnath
80,Chandigarh Transport Undertaking (CTU) - 165855,HVAC Seater (2+3),17:00,23:00,06h 00m,INR 542.8,43 Seats available,4.5\n30,https://www.redbus.in/bus-tickets/narnaul-to-c...,Narnaul to Chandigarh
82,Chandigarh Transport Undertaking (CTU) - 165763,HVAC Seater (2+3),17:00,22:50,05h 50m,INR 433.4,46 Seats available,3.4\n35,https://www.redbus.in/bus-tickets/amritsar-to-...,Amritsar to Chandigarh


# Himachal Pradesh Bus Details

In [64]:
# read the csv file
df_6=pd.read_csv("df_H.csv")
df_6

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
2,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
3,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
5,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...
6,Chamba (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/chamba-himac...
7,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
8,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Delhi to Chamba (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-cha...


In [65]:
# retrive the bus details
driver_H = webdriver.Chrome()
Bus_names_H = []
Bus_types_H = []
Start_Time_H = []
End_Time_H = []
Ratings_H = []
Total_Duration_H = []
Prices_H = []
Seats_Available_H = []
Route_names_H = []
Route_links_H = []
for i,r in df_6.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_H.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_H.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
         
    scrolling = True
    while scrolling:
        old_page_source = driver_H.page_source
        
        ActionChains(driver_H).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_H.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_H.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_H.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_H.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_H.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_H.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_H.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_H.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_H.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_H.append(bus.text)
        Route_links_H.append(link)
        Route_names_H.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_H.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_H.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_H.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_H.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_H.append(ratings.text)
    for price_elem in price:
        Prices_H.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_H.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [66]:
# from list to convert data frame
data_7 = {
    'Bus_name': Bus_names_H,
    'Bus_type': Bus_types_H,
    'Start_time': Start_Time_H,
    'End_time': End_Time_H,
    'Total_duration': Total_Duration_H,
    'Price': Prices_H,
    "Seats_Available":Seats_Available_H,
    "Ratings":Ratings_H,
    'Route_link': Route_links_H,
    'Route_name': Route_names_H
}

df_buses_7 = pd.DataFrame(data_7)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_7.csv"
df_buses_7.to_csv(path,index=False)

In [67]:
df_buses_7

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),23:30,08:00,08h 30m,617,22 Seats available,4.6\n794,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
1,Ram Dalal Holidays,Mercedes Multi-Axle Semi Sleeper (2+2),23:30,07:40,08h 10m,INR 620,15 Seats available,4.5\n593,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
2,Bedi Travels,Volvo A/C Semi Sleeper (2+2),23:00,08:00,09h 00m,477,16 Seats available,4.8\n889,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
3,,,,,,,,,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
4,,,,,,,,,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
...,...,...,...,...,...,...,...,...,...,...
163,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),23:55,05:59,06h 04m,569,22 Seats available,4.5\n153,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Dharamshala (Himachal Pradesh)
164,HRTC - 531,Ordinary 3+2 Non AC Seater,17:50,10:00,16h 10m,INR 876,34 Seats available,3.5\n43,https://www.redbus.in/bus-tickets/delhi-to-cha...,Delhi to Chamba (Himachal Pradesh)
165,HRTC - 590,A/C Executive (2+3),19:00,10:15,15h 15m,INR 1062,40 Seats available,2.8\n67,https://www.redbus.in/bus-tickets/delhi-to-cha...,Delhi to Chamba (Himachal Pradesh)
166,HRTC - 84,Himsuta AC Seater Volvo/Scania 2+2,20:15,11:30,15h 15m,INR 1704,25 Seats available,4.0\n81,https://www.redbus.in/bus-tickets/delhi-to-cha...,Delhi to Chamba (Himachal Pradesh)


In [68]:
import pandas as pd

# 1. Load DataFrame
df_buses_7 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_7.csv")

# 2. Filter for HRTC Buses
df_hrtc = df_buses_7[
    df_buses_7['Bus_name'].str.contains('HRTC', case=False, na=False) |
    df_buses_7['Bus_type'].str.contains('HRTC', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered HRTC Buses:")
print(df_hrtc.to_string(index=False))  # Display in a formatted way

# 4. Save the Filtered DataFrame to a New CSV File
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\hrtc_buses.csv"
df_hrtc.to_csv(filtered_path, index=False)

print(f"Filtered HRTC buses saved to: {filtered_path}")


Filtered HRTC Buses:
     Bus_name                           Bus_type Start_time End_time Total_duration    Price    Seats_Available Ratings                                                                Route_link                                Route_name
HRTC - 401670         Ordinary 3+2 Non AC Seater      18:00    22:00        04h 00m  INR 310 31 Seats available 3.3\n24 https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh Chandigarh to Hamirpur (Himachal Pradesh)
  HRTC - 1549         Ordinary 3+2 Non AC Seater      18:15    01:00        06h 45m  INR 362 37 Seats available  4.8\n3 https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh Chandigarh to Hamirpur (Himachal Pradesh)
  HRTC - 1001                A/C Executive (2+3)      17:15    04:00        10h 45m  INR 692 42 Seats available 3.5\n21   https://www.redbus.in/bus-tickets/chamba-himachal-pradesh-to-chandigarh   Chamba (Himachal Pradesh) to Chandigarh
   HRTC - 100 Himsuta AC Seater Vol

# Assam Bus Details

In [84]:
# read the csv file
df_7=pd.read_csv("df_AS.csv")
df_7

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
3,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
6,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
7,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...
8,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
9,North Lakhimpur to Sibsagar,https://www.redbus.in/bus-tickets/north-lakhim...


In [85]:
# retrive the bus details
driver_AS = webdriver.Chrome()
Bus_names_AS = []
Bus_types_AS = []
Start_Time_AS = []
End_Time_AS = []
Ratings_AS = []
Total_Duration_AS = []
Prices_AS = []
Seats_Available_AS = []
Route_names_AS = []
Route_links_AS = []
for i,r in df_7.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]
# Loop through each link
    driver_AS.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_AS.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
         
    scrolling = True
    while scrolling:
        old_page_source = driver_AS.page_source
        
        ActionChains(driver_AS).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_AS.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_AS.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_AS.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_AS.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_AS.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_AS.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_AS.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_AS.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_AS.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_AS.append(bus.text)
        Route_links_AS.append(link)
        Route_names_AS.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_AS.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_AS.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_AS.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_AS.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_AS.append(ratings.text)
    for price_elem in price:
        Prices_AS.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_AS.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [86]:
# from list to convert data frame
data_8 = {
    'Bus_name': Bus_names_AS,
    'Bus_type': Bus_types_AS,
    'Start_time': Start_Time_AS,
    'End_time': End_Time_AS,
    'Total_duration': Total_Duration_AS,
    'Price': Prices_AS,
    "Seats_Available":Seats_Available_AS,
    "Ratings":Ratings_AS,
    'Route_link': Route_links_AS,
    'Route_name': Route_names_AS
}

df_buses_8 = pd.DataFrame(data_8)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_8.csv"
df_buses_8.to_csv(path,index=False)

In [87]:
df_buses_8

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:45,03:15,05h 30m,INR 550,2 Seats available,3.6\n28,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
1,Shopuris Travels,Non AC Seater (2+1),23:00,04:00,05h 00m,INR 700,2 Seats available,New,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
2,Kalpana Travels,AC Seater (2+2),16:50,20:15,03h 25m,342,28 Seats available,4.8\n66,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
3,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,00:00,04h 00m,400,16 Seats available,3.6\n39,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
4,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:30,01:00,04h 30m,400,17 Seats available,4.2\n10,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
5,Pranjit Travels (Under ASTC),A/C Seater / Sleeper (2+1),20:45,02:30,05h 45m,INR 500,19 Seats available,3.3\n2,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
6,Pranjit Travels (Under ASTC),,21:15,01:45,04h 30m,INR 450,,3.1,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
7,DESTINY TRAVELS,NON A/C Seater (2+1),17:45,21:30,03h 45m,412,35 Seats available,2.4,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Nagaon (Assam)
8,Priyanka,NON A/C Seater (2+1),19:00,05:30,10h 30m,INR 600,3 Seats available,2.7\n23,https://www.redbus.in/bus-tickets/jorhat-to-no...,Jorhat to North Lakhimpur
9,MAA MONOSHA TRAVELS,NON A/C Seater (2+1),20:00,03:30,07h 30m,INR 350,25 Seats available,3.5\n5,https://www.redbus.in/bus-tickets/dhubri-to-gu...,Dhubri to Guwahati


In [88]:
import pandas as pd

# 1. Load DataFrame for Assam buses
df_buses_8 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_8.csv")

# 2. Filter for ASTC Buses
df_astc = df_buses_8[
    df_buses_8['Bus_name'].str.contains('ASTC', case=False, na=False) |
    df_buses_8['Bus_type'].str.contains('ASTC', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered ASTC Buses:")
print(df_astc.to_string(index=False))  # Display in a formatted way

# 4. Save the Filtered DataFrame to a New CSV File
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\astc_buses.csv"
df_astc.to_csv(filtered_path, index=False)

print(f"Filtered ASTC buses saved to: {filtered_path}")


Filtered ASTC Buses:
                    Bus_name                   Bus_type Start_time End_time Total_duration   Price    Seats_Available Ratings                                           Route_link         Route_name
Pranjit Travels (Under ASTC) A/C Seater / Sleeper (2+1)      20:45    02:30        05h 45m INR 500 19 Seats available  3.3\n2 https://www.redbus.in/bus-tickets/guwahati-to-tezpur Guwahati to Tezpur
Pranjit Travels (Under ASTC)                        NaN      21:15    01:45        04h 30m INR 450                NaN     3.1 https://www.redbus.in/bus-tickets/guwahati-to-tezpur Guwahati to Tezpur
Pranjit Travels (Under ASTC) A/C Seater / Sleeper (2+1)      20:45    02:30        05h 45m INR 500 19 Seats available  3.3\n2 https://www.redbus.in/bus-tickets/guwahati-to-tezpur Guwahati to Tezpur
Pranjit Travels (Under ASTC)                        NaN      21:15    01:45        04h 30m INR 450                NaN     3.1 https://www.redbus.in/bus-tickets/guwahati-to-tezpur Guwahati

# Uttra Pradesh bus details

In [89]:
# read the csv file
df_8=pd.read_csv("df_UP.csv")
df_8

,Route_name,Route_link
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...
1,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
2,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
3,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
4,Lucknow to Allahabad,https://www.redbus.in/bus-tickets/lucknow-to-a...
5,Lucknow to Delhi,https://www.redbus.in/bus-tickets/lucknow-to-d...
6,Delhi to Farrukhabad (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-far...
7,Farrukhabad (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/farrukhabad-...
8,Badaun to Delhi,https://www.redbus.in/bus-tickets/badaun-to-delhi
9,Allahabad to Lucknow,https://www.redbus.in/bus-tickets/allahabad-to...


In [90]:
# retrive the bus details
driver_UP = webdriver.Chrome()
Bus_names_UP = []
Bus_types_UP = []
Start_Time_UP = []
End_Time_UP = []
Ratings_UP = []
Total_Duration_UP = []
Prices_UP = []
Seats_Available_UP = []
Route_names = []
Route_links = []

for i,r in df_8.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_UP.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_UP.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2) 
        
    scrolling = True
    while scrolling:
        old_page_source = driver_UP.page_source
        
        ActionChains(driver_UP).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_UP.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_UP.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_UP.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_UP.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_UP.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_UP.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_UP.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_UP.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_UP.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_UP.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_UP.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_UP.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_UP.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_UP.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_UP.append(ratings.text)
    for price_elem in price:
        Prices_UP.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_UP.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [91]:
# from list to convert data frame
data_9 = {
    'Bus_name': Bus_names_UP,
    'Bus_type': Bus_types_UP,
    'Start_time': Start_Time_UP,
    'End_time': End_Time_UP,
    'Total_duration': Total_Duration_UP,
    'Price': Prices_UP,
    "Seats_Available":Seats_Available_UP,
    "Ratings":Ratings_UP,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_9 = pd.DataFrame(data_9)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_9.csv"
df_buses_9.to_csv(path,index=False)

In [92]:
df_buses_9

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,New Khaira Transport,A/C Seater / Sleeper (2+1),20:40,03:30,06h 50m,585,17 Seats available,4.2\n14,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
1,RS YADAV SMART BUS,Bharat Benz A/C Seater /Sleeper (2+1),21:40,04:45,07h 05m,INR 699,26 Seats available,4.7\n8,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
2,FlixBus,AC Seater (2+2),23:35,05:00,05h 25m,INR 440.18,8 Seats available,3.9\n73,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
3,FlixBus,Bharat Benz A/C Seater (2+2),17:15,22:25,05h 10m,INR 193.75,25 Seats available,4.6\n20,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
4,SHRI RAM TRAVELS,A/C Seater (2+2),23:59,05:15,05h 16m,INR 740,16 Seats available,3.5\n24,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
...,...,...,...,...,...,...,...,...,...,...
261,UPSRTC - ALM0024,Volvo AC Seater 2+2,18:30,23:00,04h 30m,INR 658,39 Seats available,3.3\n3,https://www.redbus.in/bus-tickets/allahabad-to...,Allahabad to Lucknow
262,UPSRTC - PYD0158,Janrath AC Seater 2+3,19:01,23:31,04h 30m,INR 396,48 Seats available,3.3\n2,https://www.redbus.in/bus-tickets/allahabad-to...,Allahabad to Lucknow
263,UPSRTC - ALM0026,Volvo AC Seater 2+2,19:30,23:59,04h 29m,INR 658,39 Seats available,3.3\n3,https://www.redbus.in/bus-tickets/allahabad-to...,Allahabad to Lucknow
264,UPSRTC - ALM0060,Volvo AC Seater 2+2,20:30,01:00,04h 30m,INR 658,39 Seats available,3.3\n2,https://www.redbus.in/bus-tickets/allahabad-to...,Allahabad to Lucknow


In [93]:
import pandas as pd

# 1. Load DataFrame
df_buses_9 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_9.csv")

# 2. Filter for UPSRTC Buses
df_upsrtc = df_buses_9[
    df_buses_9['Bus_name'].str.contains('UPSRTC', case=False, na=False) |
    df_buses_9['Bus_type'].str.contains('UPSRTC', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered UPSRTC Buses:")
print(df_upsrtc.to_string(index=False))  # Display in a formatted way

# 4. Save the Filtered DataFrame to a New CSV File
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\upsrtc_buses.csv"
df_upsrtc.to_csv(filtered_path, index=False)

print(f"Filtered UPSRTC buses saved to: {filtered_path}")


Filtered UPSRTC Buses:
        Bus_name                   Bus_type Start_time End_time Total_duration   Price    Seats_Available Ratings                                                       Route_link                       Route_name
UPSRTC - BDV0462      Janrath AC Seater 2+2      17:59    21:01        03h 02m INR 305 12 Seats available  4.0\n9 https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi Aligarh (uttar pradesh) to Delhi
UPSRTC - BDV0092 Ordinary Non AC Seater 2+3      17:59    21:18        03h 19m INR 204 52 Seats available  3.3\n1 https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi Aligarh (uttar pradesh) to Delhi
UPSRTC - BDV0096 Ordinary Non AC Seater 2+3      18:29    21:49        03h 20m INR 204 52 Seats available  3.3\n1 https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi Aligarh (uttar pradesh) to Delhi
UPSRTC - MNP0114 Ordinary Non AC Seater 2+3      18:55    21:47        02h 52m INR 255 52 Seats available     3.3 https://www.red

# West Bengal Bus Details

In [94]:
# read the csv file
df_9=pd.read_csv("df_WB.csv")
df_9

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
4,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
5,Kolkata to Suri,https://www.redbus.in/bus-tickets/kolkata-to-suri
6,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
7,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
8,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...


In [95]:
# retrive the bus details
driver_WB = webdriver.Chrome()
Bus_names_WB = []
Bus_types_WB = []
Start_Time_WB = []
End_Time_WB = []
Ratings_WB = []
Total_Duration_WB = []
Prices_WB = []
Seats_Available_WB = []
Route_names = []
Route_links = []

for i,r in df_9.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]
# Loop through each link
    driver_WB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_WB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2) 
        
    scrolling = True
    while scrolling:
        old_page_source = driver_WB.page_source
        
        ActionChains(driver_WB).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_WB.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_WB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_WB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_WB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_WB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_WB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_WB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_WB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_WB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_WB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_WB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_WB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_WB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_WB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_WB.append(ratings.text)
    for price_elem in price:
        Prices_WB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_WB.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [96]:
# from list to convert data frame
data_10 = {
    'Bus_name': Bus_names_WB,
    'Bus_type': Bus_types_WB,
    'Start_time': Start_Time_WB,
    'End_time': End_Time_WB,
    'Total_duration': Total_Duration_WB,
    'Price': Prices_WB,
    "Seats_Available":Seats_Available_WB,
    "Ratings":Ratings_WB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_10 = pd.DataFrame(data_10)
#convert dataframe to csv
path=r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_10.csv"
df_buses_10.to_csv(path,index=False)

In [97]:
df_buses_10

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,23:55,05h 20m,450,5 Seats available,1.1\n12,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
1,Satya Paribahan,A/C Seater (2+3),23:00,05:10,06h 10m,INR 420,15 Seats available,2.5\n33,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
2,Snemita Paribahan(Shinjini),A/C Seater (2+3),22:45,04:10,05h 25m,INR 420,1 Seat available,3.7\n30,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Calcutta
3,City Express (Shinjini),AC Seater (2+3),22:45,04:10,05h 25m,INR 420,16 Seats available,3.3\n33,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Calcutta
4,Satya Paribahan,A/C Seater (2+3),23:00,03:45,04h 45m,INR 420,15 Seats available,2.9\n30,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Calcutta
5,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,23:00,04h 25m,450,5 Seats available,2.0\n14,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Calcutta
6,Parameswar Travels,A/C Seater Push Back (2+3),18:10,23:15,05h 05m,285,29 Seats available,2.4\n14,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Calcutta
7,Shyamoli Paribahan Pvt Ltd,VE A/C Seater (2+2),23:30,04:30,05h 00m,INR 499,10 Seats available,4.5\n346,https://www.redbus.in/bus-tickets/kolkata-to-d...,Kolkata to Digha
8,Mayuri Express,AC Seater (2+2),23:15,04:30,05h 15m,270,27 Seats available,4.3\n193,https://www.redbus.in/bus-tickets/kolkata-to-d...,Kolkata to Digha
9,Maa Chandi Travels,A/C Seater / Sleeper (2+2),22:00,05:30,07h 30m,INR 350,34 Seats available,4.2\n220,https://www.redbus.in/bus-tickets/kolkata-to-d...,Kolkata to Digha


In [98]:
import pandas as pd

# 1. Load DataFrame
df_buses_10 = pd.read_csv(r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\df_buses_10.csv")

# 2. Filter for WBTC, SBSTC, and NBSTC Buses
df_west_bengal = df_buses_10[
    df_buses_10['Bus_name'].str.contains('WBTC|SBSTC|NBSTC', case=False, na=False) |
    df_buses_10['Bus_type'].str.contains('WBTC|SBSTC|NBSTC', case=False, na=False)
]

# 3. Display the filtered DataFrame
print("Filtered West Bengal State Transport Buses:")
print(df_west_bengal.to_string(index=False))  # Display in a formatted way

# 4. Save the Filtered DataFrame to a New CSV File
filtered_path = r"C:\Users\ajant\OneDrive\Desktop\redbus-data-scraping\Redbus\west_bengal_buses.csv"
df_west_bengal.to_csv(filtered_path, index=False)

print(f"Filtered West Bengal State Transport buses saved to: {filtered_path}")


Filtered West Bengal State Transport Buses:
                                          Bus_name            Bus_type Start_time End_time Total_duration   Price    Seats_Available Ratings                                        Route_link      Route_name
SBSTC-KOLKATA - SURI - 18:00 - (Suri Depot) - 1205 Non AC Seater (2+3)      18:00    23:00        05h 00m INR 190 51 Seats available 3.2\n18 https://www.redbus.in/bus-tickets/kolkata-to-suri Kolkata to Suri
SBSTC-KOLKATA - SURI - 18:00 - (Suri Depot) - 1205 Non AC Seater (2+3)      18:00    23:00        05h 00m INR 190 51 Seats available 3.2\n18 https://www.redbus.in/bus-tickets/kolkata-to-suri Kolkata to Suri
SBSTC-KOLKATA - SURI - 18:00 - (Suri Depot) - 1205 Non AC Seater (2+3)      18:00    23:00        05h 00m INR 190 51 Seats available 3.2\n18 https://www.redbus.in/bus-tickets/kolkata-to-suri Kolkata to Suri
SBSTC-KOLKATA - SURI - 18:00 - (Suri Depot) - 1205 Non AC Seater (2+3)      18:00    23:00        05h 00m INR 190 51 Seats avail